In [8]:
#On réunira ici toutes les déclarations de variables globales
#On importera également nos bibliothèques ici

import numpy as np
import random as rd
import copy as copy
import uuid as uuid

#type(n) : int
#n est la taille de la matrice carrée
n = 100

#type(probabilite_presence_carotte) : float [0.00 ; 1.00]
#probabilite_presence_carotte est la probabilité qu'il n'y ait pas de carottes sur une case
probabilite_presence_carotte = 0.5

#type(esperance_vie_renard) : int
#esperance_vie_renard est un nb de jours
esperance_vie_renard = 1825 #environ 5 ans

#type(esperance_vie_lapin) : int
#esperance_vie_lapin est un nb de jours
esperance_vie_lapin = 2555 #environ 7 ans

#type(efficacite_renard) : float
#efficacite_renard est l'efficacité de chasse du renard, i.e. sa chance d'attraper le lapin
efficacite_renard_generale = 0.20 #basé sur des données réelles
def efficacite_renard_specifique():
    return rd.uniform(-0.1,0.1)

#type(efficacite_lapin) : float
#efficacite_lapin est un pondérateur fictif pour éviter un système mathématique trop prédominant, c'est le facteur chance naturel
def efficacite_lapin_specifique():
    return rd.uniform(-0.1,0.1)

#type(nb_renards_initial) : int
#nb_renards_initial le nombre initial de renards
nb_renards_initial = 500

#type(nb_lapins_initial) : int
#nb_lapins_initital le nombre initial de lapins
nb_lapins_initial = 1000

#type(M) : int
#M le range de l'entourage de Moore
M = 1

#type(maturite_sexuelle_renard) : int
#maturite_sexuelle_renard le nb de jours pour pouvoir s'accoupler
maturite_sexuelle_renard = 310 #donnée réelle, correspond à 10 mois

#type(temps_gestation_renard) : int
#temps_gestation_renard le nb de jours avant de mettre bas
temps_gestation_renard = 55 #donnée réelle

# Agents

### Renard

In [9]:
def creation_renard():
    ID = uuid.uuid4()
    age = rd.randint(0,esperance_vie_renard)
    sexe = rd.choice([True, False]) #True correspond à femelle
    efficacite = efficacite_renard_generale + efficacite_renard_specifique()
    fecondite = rd.uniform(0.2,1.0)
    position = (rd.randint(0,n-1),rd.randint(0,n-1))
    faim = rd.random()
    excitation = rd.random()
    gestationnaire = (False, 0, 0) #en gestation ou non, nb de jour avant mis à bas, nb de renardeaux
    type_agent = "renard"
    
    return (ID, (type_agent, (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position)))

In [10]:
print(creation_renard())

(UUID('9d307ab6-1680-4fe6-9326-b0bf8e2e5281'), ('renard', (1764, True, 0.2558204775031682, 0.2034681771617053, 0.9062506100163046, 0.8451975075913843, (False, 0, 0), (90, 79))))


In [11]:
def creation_renards():
    cpt = 0 #type(cpt) : int
    renards = dict()
    while cpt < nb_renards_initial:
        cpt += 1
        identifiant, renard = creation_renard()        
        renards[identifiant] = renard
    return renards

In [12]:
print(creation_renards())

{UUID('cadef411-4902-49d0-a55a-1dcb2a222d4b'): ('renard', (1663, True, 0.1421878192419016, 0.8764002547601808, 0.23941997289697647, 0.2124443494593492, (False, 0, 0), (46, 86))), UUID('ef0555a7-6616-4a4c-bc06-659bed195562'): ('renard', (1738, True, 0.2993207776531424, 0.6253098113615836, 0.983437312560565, 0.11576585824410368, (False, 0, 0), (12, 94))), UUID('b92678c9-31b5-4b39-add6-210751d3460b'): ('renard', (1382, True, 0.2935739568745105, 0.42086351028927205, 0.5484504136844001, 0.1806065873564343, (False, 0, 0), (62, 81))), UUID('f6fc158a-1dda-4ff1-8fd8-07b6ed163637'): ('renard', (1255, True, 0.1181304856973277, 0.6623154783612414, 0.840022598176491, 0.0833179304419116, (False, 0, 0), (70, 5))), UUID('f8b099bc-026e-4feb-82f6-ef65584e8c72'): ('renard', (434, False, 0.2262514856650257, 0.33804472605167313, 0.8799230803061326, 0.3431893556157156, (False, 0, 0), (50, 78))), UUID('37d69103-3f4e-4e33-a061-d227d7498264'): ('renard', (697, True, 0.11114686842140042, 0.7439861452690568, 0.0

### Lapin

In [88]:
def creation_lapin():
    """None -> lapin
    lapin : tuple[int, bool, float, float, float, float, tuple[bool, int], tuple[int, int]]"""
    ID = uuid.uuid4()
    age = rd.randint(0, esperance_vie_lapin)
    sexe = rd.choice([True, False]) #True correspond à femelle
    efficacite = efficacite_lapin_specifique()
    fecondite = rd.uniform(0.2,1.0)
    position = (rd.randint(0,n-1),rd.randint(0,n-1))
    faim = rd.random()
    excitation = rd.random()
    gestationnaire = (False, 0, 0) #en gestation ou non, nb de jour avant mis à bas, nb de lapinous
    type_agent = "lapin"
    
    return (ID, (type_agent, (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position)))

In [14]:
print(creation_lapin())

(UUID('d88d546e-a57f-4543-81e2-56aebabc5f40'), ('lapin', (262, False, 0.06010616060964069, 0.27932239450762514, 0.25265283693937324, 0.1172037256317272, (False, 0), (31, 90))))


In [15]:
def creation_lapins():
    cpt = 0 #type(cpt) : int
    lapins = dict()
    while cpt < nb_renards_initial:
        cpt += 1
        identifiant, lapin = creation_lapin()        
        lapins[identifiant] = lapin
    return lapins

In [16]:
print(creation_lapins())

{UUID('918b912f-a6fc-46e8-9754-235be767d497'): ('lapin', (2276, True, 0.014484462517742575, 0.8595409257685989, 0.7450811758227172, 0.0861517215117491, (False, 0), (22, 3))), UUID('b45660e5-e432-455c-8b2f-59cd2d0e20f1'): ('lapin', (1668, True, 0.011418615706528995, 0.6531357083215716, 0.6498526175303122, 0.3949225451871773, (False, 0), (27, 5))), UUID('340dad70-baee-48ca-bc89-b984bacd71a1'): ('lapin', (1253, True, 0.006916652699992781, 0.44649545388535067, 0.622974884964328, 0.5848835112145571, (False, 0), (40, 45))), UUID('56588873-8504-4448-8d0a-18e32680fdae'): ('lapin', (1600, False, 0.05235635541479894, 0.2737239480322575, 0.18229447577364966, 0.5531619312621345, (False, 0), (87, 11))), UUID('09971abe-c1e5-423f-898d-071940627e0c'): ('lapin', (1035, True, -0.0992603592861864, 0.9903821427690789, 0.9675200332170187, 0.159489947995753, (False, 0), (64, 98))), UUID('ee891881-97d6-44ae-9d82-4e5346dcfa60'): ('lapin', (472, True, 0.045690810269680304, 0.9369301134418326, 0.359926956986603

### Carotte

In [39]:
def creation_carotte(ligne, colonne):
    return (uuid.uuid4(), ("carotte", (rd.random(),(ligne, colonne))))

In [82]:
print(creation_carotte(0,0))

(UUID('24858171-fe74-40ea-984c-a72842e00c39'), ('carotte', (0.9344335384121768, (0, 0))))


In [72]:
def creation_carottes():
    cpt = 0
    carottes = dict()
    while cpt < n*n:
        cpt += 1
        if rd.random() >= probabilite_presence_carotte:
            identifiant, carotte = creation_carotte((cpt-(cpt%100))//100,cpt%100)
            carottes[identifiant] = carotte
    return carottes

In [68]:
print(creation_carottes())

{UUID('416492e2-c820-4c4f-b5b9-0122be591d73'): ('carotte', (0.6553263711113119, (85, 56))), UUID('188178e9-d9c6-45a9-88e4-bdea82c4f2b2'): ('carotte', (0.1399195499098873, (68, 15))), UUID('e76e62ef-7869-47fd-b14f-27eb66baa304'): ('carotte', (0.3017230003493726, (58, 63))), UUID('f04bf8d0-f7ab-48e0-8f7b-2de1f3dd78c8'): ('carotte', (0.4893965000107726, (1, 39))), UUID('ff03efa5-e88e-4a78-a336-39fc1c12d768'): ('carotte', (0.15415098032147012, (75, 31))), UUID('01d1dc45-ea72-402f-a195-91fcc794be8d'): ('carotte', (0.3974404966779338, (42, 45))), UUID('6c2fbd04-594b-4001-b8c8-37d0827d9fe4'): ('carotte', (0.7935252304896189, (85, 64))), UUID('b89a5cb4-860e-4c97-948f-9725fcadbb23'): ('carotte', (0.7755256750667374, (40, 96))), UUID('521d7840-0dea-4d64-8fa6-5902e53314d5'): ('carotte', (0.5917409773632529, (94, 90))), UUID('feb566bf-4ce8-4cc7-ba24-2caa283af994'): ('carotte', (0.8846170652241877, (61, 16))), UUID('7bfc1929-b302-4ebe-96a2-65d8627849ff'): ('carotte', (0.4153171905564649, (0, 92))),

### Mélange des agents

On déclare en variable globale le dictionnaire rassemblant tous nos agents

In [73]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [74]:
agents = merge_two_dicts(creation_lapins(), creation_renards())
carottes = creation_carottes()

In [75]:
print(agents)
print(carottes)

{UUID('5eff936c-d8b1-4ac0-b2aa-e8f282cb81e6'): ('renard', (134, False, 0.1655762068842751, 0.9501990764890809, 0.8135638916486273, 0.2997328564962485, (False, 0, 0), (72, 85))), UUID('8b6c5902-c453-4c72-a8d4-7a8dd88d144c'): ('renard', (430, False, 0.18308778273642246, 0.9690208735209238, 0.3017258665972916, 0.3023364224972177, (False, 0, 0), (94, 71))), UUID('ae8f1775-3f6d-408c-83e9-b4cb9b2cb378'): ('renard', (1158, False, 0.16284401650571503, 0.4375576637744665, 0.3507796390971589, 0.22341127484037893, (False, 0, 0), (86, 49))), UUID('e039b1a6-fff5-4e65-9fdc-0f52ac2934a4'): ('lapin', (405, False, -0.017723283393591235, 0.8934112357893149, 0.34923724288575575, 0.22655703162740504, (False, 0), (85, 73))), UUID('234896cf-d5e4-45cc-bc0d-c6923a898874'): ('renard', (134, True, 0.21993415412333278, 0.5293435638458086, 0.24861705954666824, 0.003105870628152041, (False, 0, 0), (67, 52))), UUID('4ee87092-20cf-41e9-9fc0-d5266f7748ac'): ('renard', (422, False, 0.26503319517914325, 0.6480091983607

# Environnement

### Initialisation de l'environnement

In [24]:
def init_matrice():
    a = np.empty((n,n), dtype=np.object_)
    a.fill([])
    a = np.frompyfunc(list,1,1)(a)
    return a

In [25]:
environnement = init_matrice()

In [26]:
print(environnement)

[[list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 ...
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]]


In [27]:
test = init_matrice()
test[0,1].append("banane")
print(test)

[[list([]) list(['banane']) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 ...
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]]


### Placement des agents dans l'environnement

In [30]:
def update_matrice():
    for e in agents:
        _,agent = agents[e]
        _,_,_,_,_,_,_,position = agent
        ligne, colonne = position
        environnement[ligne, colonne].append(e)
    return environnement

In [31]:
print(update_matrice())

[[list([]) list([]) list([]) ... list([])
  list([UUID('c8ab1b09-bba4-4ce3-bb18-e7d5ab1a67cb'), UUID('c8ab1b09-bba4-4ce3-bb18-e7d5ab1a67cb')])
  list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([])
  list([UUID('c3a4a172-99b8-471f-bff9-099651f587b9'), UUID('c3a4a172-99b8-471f-bff9-099651f587b9')])
  ...
  list([UUID('f5fa8a19-dc76-402e-8aad-a898fd06ef3a'), UUID('f5fa8a19-dc76-402e-8aad-a898fd06ef3a')])
  list([])
  list([UUID('36afb740-ec2c-4545-97a6-7d1cee2d6293'), UUID('36afb740-ec2c-4545-97a6-7d1cee2d6293')])]
 ...
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]
 [list([]) list([]) list([]) ... list([]) list([]) list([])]]


In [79]:
def uptade_matrice_carotte():
    for e in carottes:
        _,carotte = carottes[e]
        _,position = carotte
        ligne, colonne = position 
        environnement[ligne, colonne].append(e)
    return environnement

In [80]:
print(uptade_matrice_carotte())

[[list([UUID('1f087014-eb36-4d14-9010-a7931615c99e'), UUID('1f087014-eb36-4d14-9010-a7931615c99e')])
  list([])
  list([UUID('9525c1f8-0d5d-4a5e-af34-5d5995fa4b56'), UUID('9525c1f8-0d5d-4a5e-af34-5d5995fa4b56'), UUID('9525c1f8-0d5d-4a5e-af34-5d5995fa4b56')])
  ...
  list([UUID('814df90e-3efd-4355-9e43-c353fea17487'), UUID('814df90e-3efd-4355-9e43-c353fea17487'), UUID('814df90e-3efd-4355-9e43-c353fea17487')])
  list([UUID('c8ab1b09-bba4-4ce3-bb18-e7d5ab1a67cb'), UUID('c8ab1b09-bba4-4ce3-bb18-e7d5ab1a67cb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6f-dc034730b4eb'), UUID('f8bd14ee-3a2d-4681-9e6

### Interaction des agents

In [84]:
def reussite_chasse(efficacite_renard,efficacite_lapin):
    """donne un booléen qui correspond à l'issue de la chasse du renard"""
    return rd.random()<(efficacite_renard-efficacite_lapin)

In [87]:
def nb_renardeaux(age3,age4,fec3,fec4):
    """donne le nombre de renardeaux dans une portée aléatoirement"""
    #d:int
    d=0
    if age3>310 and age4>310:
        if (fec3+fec4)/2>=0.5:
            d=rd.randint(4,5)
    return d

In [ ]:
def mouvement_renard(id_renard):
    
    _,renard = agents(id_renard)
    age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position = renard
    ligne, colonne = position
    en_gestation,jour,nb = gestationnaire
    
    entourage_moore = [] #liste des uuid aux environs
    
    for a in range(ligne-M,ligne+M):
        for b in range(colonne-M,colonne+M):
            for c in environnement[a,b]:
                entourage_moore.append(c)
    
    #On retire les carottes de l'entourage
    entourage_moore_renard = []
    entourage_moore_lapin = []
    
    for e in entourage_moore:
        type_agent,_ = agents[e]
        if type_agent = "renard":
            entourage_moore_renard.append(e)
        elif type_agent = "lapin":
            entourage_moore_lapin.append(e)        
    
    #On examine si le renard veut manger ou se reproduire
    if (faim >= excitation and len(entourage_moore_lapin) != 0) or (faim < excitation and len(entourage_moore_lapin) != 0 and len(entourage_moore_renard)==0):
        id_proie = rd.choice(entourage_moore_lapin)
        _,proie = agents[id_proie]
        _,_,efficacite_proie,_,_,_,_, position_proie = proie
        ligne_proie, colonne_proie = position_proie
        if reussite_chasse(efficacite, efficacite_proie):
            environnement[ligne_proie, colonne_proie].remove(id_proie)
            faim = 0
            agents[id_renard] = ("renard", (age, sexe, efficacite, fecondite, excitation, 0, gestationnaire, position))
        environnement[ligne, colonne].remove(id_renard)
        environnement[ligne_proie, colonne_proie].append(id_renard)
    elif (faim< excitation and len(entourage_moore_renard) != 0) or (faim >= excitation and len(entourage_moore_lapin) == 0 and len(entourage_moore_renard) !=0):
        id_proie = rd.choice(entourage_moore_renard)
        _,proie = agents[id_proie]
        age_proie,sexe_proie,_,fecondite_proie,_,_,gestationnaire_proie,position_proie = proie
        en_gestation_proie,jour_proie,nb_proie = gestationnaire_proie
        ligne_proie, colonne_proie = position_proie
        
        
        if reussite_chasse(efficacite, efficacite_proie):
            environnement[ligne_proie, colonne_proie].remove(id_proie)
            faim = 0
            agents[id_renard] = ("renard", (age, sexe, efficacite, fecondite, excitation, 0, gestationnaire, position))
        environnement[ligne, colonne].remove(id_renard)
        environnement[ligne_proie, colonne_proie].append(id_renard)
        